In [ ]:
import pandas as pd
import numpy as np
import dtale
import dtale.app as dtale_app
import requests
from pandas_datareader.data import DataReader
from datetime import datetime
#set pd for 2 decimal float 
pd.set_option('display.float_format', lambda x: '%.2f' % x)
#functiion for dtale 
kill=!sudo kill -9 $(sudo lsof -t -i:40000)
print('No active dtale')

instance_lst =[]
def create_dtale (df,name):
    if not (name in instance_lst) :
        dtale.show(df,name=name)
        instance_lst.append(name)
        dtale.instances()
    else :
        print('Company already exists!')
        dtale.instances() 

In [ ]:
#start=datetime(end.year - 1, end.month, end.day)
end = datetime.now()
start=datetime(end.year - 1, end.month, end.day)
symbol=input('Please Enter Stock Symbol : ').upper()
try:
    df=DataReader(symbol, 'yahoo', start, end)
    df.drop('Adj Close', axis=1,inplace=True)
    print('Done')
except:
    print("Please Check Internet Connection")
#Data Cleaning
df.reset_index(inplace=True)
df = df[['Date', 'Open', 'High', 'Low', 'Close','Volume']]
#Saving Data
df.to_csv('./Saved/'+symbol+'.csv',index=False)
df.tail(1)

In [ ]:
#Add calulated Columns 
# add calculated col Candle stick 
df['Candle'] = df.Close - df.Open
df['Ch%']=(df.Candle/df.Open)*100
#Highs percent of open price 
df['H%']=((df.High-df.Open)/df.Open)*100

#lows percent of open price
df['L%']=((df.Open-df.Low)/df.Open)*100

#High - low value 
df['HL']=df.High-df.Low
df['HLCH%']=(df.HL/df.Open)*100

#Low to close 
df['LtC']=df.Close-df.Low
df['LtC%']=(df.LtC/df.Open)*100

# Converte date col to date type then get day name by date value 
df['Date'] = pd.to_datetime(df['Date'])
df['Day'] = df['Date'].dt.day_name()
#Calculate mean , median 

# apply up trend if Candle is postive other is negative 
df['UPTrend'] = np.where(df['Candle']>0, True, False)

#Filter unwanted column
FilteredDf= df#[['Date','Open','Candle','Ch%','H%','L%','HL','HLCH%','LtC%','Day','UPTrend']]
uptrend= FilteredDf.loc[df['UPTrend'] == True]
downtrend= FilteredDf.loc[df['UPTrend'] == False]

create_dtale(df=df,name=symbol)


In [ ]:
#TimeFrame
#Spicific Date 
spec_date=df.loc[df['Date']>='2021-02-10']

#last 5 days 
LastW=df.set_index("Date").last("1W")

#Month
Last1M=df.set_index("Date").last("1M")

#6month
Last6M=df.set_index("Date").last("6M")

#Year Start
thisY=df.set_index("Date").last("1Y")
create_dtale(df=thisY,name='thisY_'+symbol)

In [ ]:
#Days
Monday=df.loc[df['Day'] == 'Monday']
Tuesday=df.loc[df['Day'] == 'Tuesday']
Wednesday=df.loc[df['Day'] == 'Wednesday']
Thursday=df.loc[df['Day'] == 'Thursday']
Friday=df.loc[df['Day'] == 'Friday']

#if previous cahnge is negative 
def trendCount(df):
    d= {'Date': [], 'Trend': [],'Ch%': []}    
    trendCount=[]
    ncount=0
    pcount=0
    for index, row in df.iterrows():
        if row['Candle']<0:
            ncount=ncount-1
            d['Date'].append(row['Date'])
            d['Trend'].append(ncount)
            d['Ch%'].append(row['Candle'])
            pcount=0
        if row['Candle']>=0:
            pcount=pcount+1
            d['Date'].append(row['Date'])
            d['Trend'].append(pcount)
            d['Ch%'].append(row['Candle'])
            ncount=0
    return d

resultdf=pd.DataFrame(trendCount(df))
resultdf.tail(30)

In [ ]:
import pandas as pd
import numpy as np
#thisY=df.set_index("Date").last("6M")
var=spec_date
Described=var.describe()
DescribedUp=var.loc[var['UPTrend'] == True].describe()
DescribedDown=var.loc[var['UPTrend'] == False].describe()
def PrintHL(Described):
    print('#High')
    print('Max {:.2f}'.format(Described['H%'].iloc[7]),' :',float(float(priceToday)*((Described['H%'].iloc[7]/100)+1)))
    print('Mid {:.2f}'.format(Described['H%'].iloc[5]),' :',float(float(priceToday)*((Described['H%'].iloc[5]/100)+1)))
    print('Mean {:.2f}'.format(Described['H%'].iloc[1]),' :',float(float(priceToday)*((Described['H%'].iloc[1]/100)+1)))
    print('Min {:.2f}'.format(Described['H%'].iloc[3]),' :' ,float(float(priceToday)*((Described['H%'].iloc[3]/100)+1)))

    print('#Low')
    print('Min {:.2f}'.format(Described['L%'].iloc[3]),' : ',float(float(priceToday)*(1-(Described['L%'].iloc[3]/100))))
    print('Mean {:.2f}'.format(Described['L%'].iloc[1]),' : ',float(float(priceToday)*(1-(Described['L%'].iloc[1]/100))))
    print('Mid {:.2f}'.format(Described['L%'].iloc[5]),' : ',float(float(priceToday)*(1-(Described['L%'].iloc[5]/100))))
    print('Max {:.2f}'.format(Described['L%'].iloc[7]),' : ', float(float(priceToday)*(1-(Described['L%'].iloc[7]/100))))
    
priceToday=float(input("Please Enter Price Today "))
print('Open Price= ',priceToday)
print('General')
PrintHL(Described)
print('#################################################')
print('UpTrend')
PrintHL(DescribedUp)
print('#################################################')
print('DownTrend')
PrintHL(DescribedDown)
